# Загрузка библиотек

In [1]:
!pip install pymorphy2 nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 21.5 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 40.2 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=4529eb0c54ef94ef02a98771b8ae565019f924a5f688bea00d9df68eedc86013
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [2]:
!mkdir vectorizers

# Предобработка данных для vectorizers

In [3]:
import pymorphy2
import pandas as pd
morph = pymorphy2.MorphAnalyzer()
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords');
stopwords = set(stopwords.words('russian'))
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
def lemmatize(text):
    words = text.split() # разбиваем текст на слова
    #print(words)
    res = list()
    for word in words:
        if (word not in stopwords and len(word) > 1):
            p = morph.parse(word)[0]
            res.append(p.normal_form)
    text = " ".join(res)
    return text

def text_cleaner(input_text):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', input_text)
    rem_url = re.sub(r'http\S+', '',cleantext)
    rem_url= re.sub(r'([a-z]{2}\d+[a-z]{2})',' ',rem_url)
    rem_url = re.sub(r'!+','.', rem_url)
    rem_num = re.sub('[0-9]+', '', rem_url)
    text = re.sub(r"[-—()\"#/@;:<>{}=~|€«»$\+'_–\*°“”\\√&×•ó÷≈„()‽\+.,!]+", " ", rem_num)
    text = re.sub("!",".", text)
    text = re.sub(r' +', ' ', text)
    text = re.sub(r"\[.+\]",'', text)
    emoji_pattern=re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U0001F1F2-\U0001F1F4"  # Macau flag
        u"\U0001F1E6-\U0001F1FF"  # flags
        u"\U0001F600-\U0001F64F"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U0001F1F2"
        u"\U0001F1F4"
        u"\U0001F620"
        u"\u200d"
        u"\u2640-\u2642"
        "]+", flags=re.UNICODE)

    text = emoji_pattern.sub('', text)
    text = text.lower()
    return text


def dataset_preprocessor_vectorizers(data, w_count_to_drop=3):
    data_frame = data.copy()

    indexes_to_drop = []
    # Дропаем дубли
    data_frame = data_frame.drop_duplicates(subset=['Текст инцидента']).reset_index(drop=True)

    # Regex для групп
    p = re.compile("\[.+\|.+\]")

    for index, row in tqdm(data_frame.iterrows()):

        # Очистка текста
        data_frame.loc[index, "Текст инцидента"] = text_cleaner(data_frame.loc[index, "Текст инцидента"])

        # Отсекаем сообщения из w_count_to_drop слов
        if len(row["Текст инцидента"].split(" ")) < w_count_to_drop and len(row["Текст инцидента"].split(".")) < w_count_to_drop:
            indexes_to_drop.append(index)

    # Дропаем индексы < w_count_to_drop слов
    #print(indexes_to_drop)
    data_frame = data_frame.drop(index=indexes_to_drop).reset_index(drop=True)

    # Лемматизация и удаление стоп слов
    for index, row in tqdm(data_frame.iterrows()):
        data_frame.loc[index, "Текст инцидента"] = lemmatize(row["Текст инцидента"])
    return data_frame



# Сохраняем предобработанный датасет

In [34]:
train_all = pd.read_csv("train_dataset_train.csv", delimiter=";")
train_all_pp = dataset_preprocessor_vectorizers(train_all)
train_all_pp.to_csv("train.csv", index=False)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

# MLFLOW

In [3]:
import pandas as pd
import mlflow
from sklearn.model_selection import train_test_split, ParameterGrid
import torch
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, f1_score, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from tqdm.notebook import tqdm

models =  [["tfidf", TfidfVectorizer, None],
           ["tfidf", TfidfVectorizer, 15000],
           ["tfidf", TfidfVectorizer, 10000],
           ["count", CountVectorizer, None],
           ["count", CountVectorizer, 15000],
           ["count", CountVectorizer, 10000]]

/opt/conda/lib/python3.10/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/pydantic/_internal/_config.py:321: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


## Создание датасетов

In [4]:
import pickle
class Model_tfidf:
    def __init__(self, vectorizer_name, vectorizer, max_features=None):
        self.max_features = max_features
        self.vectorizer = vectorizer(max_features=self.max_features)
        self.vectorizer_name = vectorizer_name

    def create_tfidf(self, train, test):
      tf = self.vectorizer.fit_transform(train)
      with open(f'vectorizers/{self.vectorizer_name}_vectorizer_{self.max_features}.pk', 'wb') as fin:
        pickle.dump(self.vectorizer, fin)
      vocab = self.vectorizer.vocabulary_
      train_tf = pd.DataFrame(tf.toarray(), columns=sorted(vocab.keys()))
      test_tf = pd.DataFrame(self.vectorizer.transform(test).toarray(), columns=sorted(vocab.keys()))
      del tf
      return train_tf, test_tf


    def create_datasets(self, train, test):
        train_, test_ = train.copy(), test.copy()

        train_tf_idf, test_tf_idf = self.create_tfidf(train_["Текст инцидента"].to_list(), test_["Текст инцидента"].to_list())

        train_ = pd.concat([train_, train_tf_idf], axis=1)
        test_ = pd.concat([test_, test_tf_idf], axis=1)
        del train_tf_idf
        del test_tf_idf
        #train_.to_csv(f"datasets/{self.vectorizer_name}_train_{self.max_features}.csv", index=False)
        #test_.to_csv(f"datasets/{self.vectorizer_name}_test_{self.max_features}.csv", index=False)

        return train_, test_

In [5]:
data = pd.read_csv("data/train_tfidif.csv")
data=data.dropna()
#mtf = Model_tfidf("count", CountVectorizer, 5000)

In [6]:
column = "Группа тем"
train, test = train_test_split(data,
                               test_size=0.33, random_state=42, stratify=data[column])

train, test = train.reset_index(drop=True), test.reset_index(drop=True)

### Тест класса Model_tfidf

In [7]:
#train, test = mtf.create_datasets(train, test)

In [19]:
#train.head(2)


,Исполнитель,Группа тем,Текст инцидента,Тема,admin,admperm,black,chaikovsky,chp,chus,...,ямский,ян,янв,январский,январь,яркий,ясно,ящик,ёлка,ёлочка
0,АО ПРО ТКО,Мусор/Свалки/ТКО,березники второй город край пункт приём,Раздельная сортировка отходов,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Министерство социального развития ПК,Социальное обслуживание и защита,добрый день выплата ребёнок быть год январь,Задержка выплат гражданам,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [8]:
param_grid = {
    'iterations': [100, 250]
}
grid = list(ParameterGrid(param_grid))

In [9]:
mlflow.set_experiment('vectorizers')

2023/11/24 12:28:13 INFO mlflow.tracking.fluent: Experiment with name 'vectorizers' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///workspace/mlruns/379540396365276748', creation_time=1700828893417, experiment_id='379540396365276748', last_update_time=1700828893417, lifecycle_stage='active', name='vectorizers', tags={}>

In [10]:
for model_ in models:
    model_name, vectorizer, max_features = model_
    model = Model_tfidf(model_name, vectorizer, max_features)

    train_, test_ = model.create_datasets(train, test)

    for params in grid:
        try:
            with mlflow.start_run(nested=True):
                catboost_model = CatBoostClassifier(**params, verbose=0, random_seed=42)
                catboost_model.fit(train_.drop(columns=["Текст инцидента", "Группа тем", "Исполнитель", "Тема"]), train_["Группа тем"])

                predictions = catboost_model.predict(test_.drop(columns=["Текст инцидента", "Группа тем", "Исполнитель", "Тема"]))

                accuracy = accuracy_score(test_["Группа тем"], predictions)
                f1 = f1_score(test_["Группа тем"], predictions, average='weighted')
                report = classification_report(test_["Группа тем"], predictions, output_dict=True)
                report_text =  classification_report(test_["Группа тем"], predictions)


                mlflow.log_metric("report_accuracy", report['accuracy'])
                mlflow.log_metric("macro avg_precision", report['macro avg']['precision'])
                mlflow.log_metric("macro avg_recall", report['macro avg']['recall'])
                mlflow.log_metric("macro avg_f1-score", report['macro avg']['f1-score'])
                mlflow.log_metric("weighted avg_precision", report['weighted avg']['precision'])
                mlflow.log_metric("weighted avg_recall", report['weighted avg']['recall'])
                mlflow.log_metric("weighted avg_f1-score", report['weighted avg']['f1-score'])

                mlflow.log_text(report_text, "classification_report.txt")

                mlflow.set_tag("vectorizer_name", model_name)
                mlflow.set_tag("max_features", max_features)
                mlflow.set_tag("dataset_name", "lemmatized_cleaned_v2")
                mlflow.set_tag("model_name", "catboost")
                mlflow.catboost.log_model(catboost_model, "model")
                mlflow.log_params(params)
                mlflow.log_metrics({'accuracy': accuracy, 'f1-weighted': f1})
        except Exception as e:
            error_name = type(e).__name__
            print(f"Caught an error: {error_name}")

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classif

In [ ]:
tfidf